In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import binascii



# Categories.csv

In [ ]:
categories = pd.read_csv('./raw_data/categories.csv')

In [26]:
print(categories.head())

   categoryID    categoryName  \
0           1       Beverages   
1           2      Condiments   
2           3     Confections   
3           4  Dairy Products   
4           5  Grains/Cereals   

                                         description  \
0            Soft drinks coffees teas beers and ales   
1  Sweet and savory sauces relishes spreads and s...   
2                  Desserts candies and sweet breads   
3                                            Cheeses   
4                   Breads crackers pasta and cereal   

                                             picture  
0  0x151C2F00020000000D000E0014002100FFFFFFFF4269...  
1  0x151C2F00020000000D000E0014002100FFFFFFFF4269...  
2  0x151C2F00020000000D000E0014002100FFFFFFFF4269...  
3  0x151C2F00020000000D000E0014002100FFFFFFFF4269...  
4  0x151C2F00020000000D000E0014002100FFFFFFFF4269...  


In [ ]:
# The "picture" column seems to be in Hex format. Let's try to convert it to an actual BMP

picture_binary = picture_binary = categories['picture'].apply(lambda x: binascii.hexlify(x[:2].encode()) if x.startswith('0x') else binascii.hexlify(x.encode()))

with open("output_image.bmp", "wb") as image_file:
    image_file.write(picture_binary[0])

# Hm, the bmp is bogus and there is actually just one string for all categories. Probably it's some kind of placeholder / mock-up then?
categories['picture'].value_counts()


# Customers.csv

In [18]:
customers = pd.read_csv('./raw_data/customers.csv')

In [19]:
customers.head()

,customerID,companyName,contactName,contactTitle,address,city,region,postalCode,country,phone,fax
0,ALFKI,Alfreds Futterkiste,Maria Anders,Sales Representative,Obere Str. 57,Berlin,NaN,12209,Germany,030-0074321,030-0076545
1,ANATR,Ana Trujillo Emparedados y helados,Ana Trujillo,Owner,Avda. de la Constitución 2222,México D.F.,NaN,05021,Mexico,(5) 555-4729,(5) 555-3745
2,ANTON,Antonio Moreno Taquería,Antonio Moreno,Owner,Mataderos 2312,México D.F.,NaN,05023,Mexico,(5) 555-3932,NaN
3,AROUT,Around the Horn,Thomas Hardy,Sales Representative,120 Hanover Sq.,London,NaN,WA1 1DP,UK,(171) 555-7788,(171) 555-6750
4,BERGS,Berglunds snabbköp,Christina Berglund,Order Administrator,Berguvsvägen 8,Luleå,NaN,S-958 22,Sweden,0921-12 34 65,0921-12 34 67


Let's try to dissect the individual columns:
- **customerID**: There seems to be no clear logic to how exactly this ID comes about. Sometimes it is the first 5 characters straight, sometimes it is three from the first word, sometimes it is four from the first word. Might be human-created. It will be quite difficult to auto-generate this ID and—if human-created—it will be tedious to figure out if a possible ID for a new customer is already present in the data or not. **We might have to create a more auto-generatable ID schema to use that as a primary key in our models later!**
- **companyName** / **contactName**: These are proper names and as such they are usually sprinkled with special characters. We need to pay attention to the encoding and maybe consider removing/transliterating some of them.
- **contactTitle**: No comments.
- **address** / **city**: Again, we need to pay attention to the special characters.
- **region**: Contains a lot of missing values. Probably some problem with how the data was created. In a real-life scenario, I would follow-up with upstream actors to clarify the inconsistency, because we know for a fact that e.g. both Germany and France do have regions.
- **postalCode**: 
- **country**: Important information for standardising address, phone, and fax data later. Maybe add a country ID later?
- **phone/fax**: To be standardised later for e.g. automatic calling from call centres or the like.

# Employees.csv

In [21]:
employees = pd.read_csv('./raw_data/employees.csv')
print(employees.head())

   employeeID   lastName firstName                 title titleOfCourtesy  \
0           1    Davolio     Nancy  Sales Representative             Ms.   
1           2     Fuller    Andrew  Vice President Sales             Dr.   
2           3  Leverling     Janet  Sales Representative             Ms.   
3           4    Peacock  Margaret  Sales Representative            Mrs.   
4           5   Buchanan    Steven         Sales Manager             Mr.   

                 birthDate                 hireDate                   address  \
0  1948-12-08 00:00:00.000  1992-05-01 00:00:00.000  507 20th Ave. E. Apt. 2A   
1  1952-02-19 00:00:00.000  1992-08-14 00:00:00.000        908 W. Capital Way   
2  1963-08-30 00:00:00.000  1992-04-01 00:00:00.000        722 Moss Bay Blvd.   
3  1937-09-19 00:00:00.000  1993-05-03 00:00:00.000      4110 Old Redmond Rd.   
4  1955-03-04 00:00:00.000  1993-10-17 00:00:00.000           14 Garrett Hill   

       city region postalCode country       homePhone  e

- **employeeID**: 
- **lastName / firstName**: Check special characters
- **birthDate / hireDate**: Look like proper dates, maybe write some range tests for plausibility of age?
- **address / city / region / postalCode / homePhone**: Normalise later
- **extension**: I don't quite know what they would be. Is it the extension of the phone number?
- **reportsTo**: Write tests for relational integrity
- **photo**: again, only 3 distinct values and very short ones indeed that I could not get to represent an actual picture.
- **photoPath**: Is not an actual URL, seems to be mockup data. Still write tests for proper URL?


# order_details.csv

In [22]:
order_details = pd.read_csv('./raw_data/order_details.csv')
print(order_details.head())

   orderID  productID  unitPrice  quantity  discount
0    10248         11       14.0        12       0.0
1    10248         42        9.8        10       0.0
2    10248         72       34.8         5       0.0
3    10249         14       18.6         9       0.0
4    10249         51       42.4        40       0.0


- **orderID**: should reference `orderID` from `orders`.
- **productID**: should reference `productID` from `products`
- **unitPrice**: should reference the `unitPrice` from `products` for the given `productID`
- **quantity**: Since `orders` does not specify the total for the each individual order we cannot verify if the quantity is correct by adding up the subtotals for each `productID` per `orderID`
- **discount**: Judging from the range between 0 and 0.26, I assume this is percentage discount. Is thus naturally capped at 1 (test this!).

# orders.csv

In [23]:
orders = pd.read_csv('./raw_data/orders.csv')
print(orders.head())

   orderID customerID  employeeID                orderDate  \
0    10248      VINET           5  1996-07-04 00:00:00.000   
1    10249      TOMSP           6  1996-07-05 00:00:00.000   
2    10250      HANAR           4  1996-07-08 00:00:00.000   
3    10251      VICTE           3  1996-07-08 00:00:00.000   
4    10252      SUPRD           4  1996-07-09 00:00:00.000   

              requiredDate              shippedDate  shipVia  freight  \
0  1996-08-01 00:00:00.000  1996-07-16 00:00:00.000        3    32.38   
1  1996-08-16 00:00:00.000  1996-07-10 00:00:00.000        1    11.61   
2  1996-08-05 00:00:00.000  1996-07-12 00:00:00.000        2    65.83   
3  1996-08-05 00:00:00.000  1996-07-15 00:00:00.000        1    41.34   
4  1996-08-06 00:00:00.000  1996-07-11 00:00:00.000        2    51.30   

                    shipName          shipAddress        shipCity shipRegion  \
0  Vins et alcools Chevalier   59 rue de l'Abbaye           Reims        NaN   
1         Toms Spezialitäten

- **orderID**: All of these should occur at least once in order_details.
- **customerID**: Should reference `customerID` from `customers`.
- **employeeID**: all of these values should be in `employeeID` in `employees`
- **requiredDate**: The date of contractual obligation to deliver, i.e. the date the customer expects the shipment the latest. If `shippedDate` > `requiredDate` , then a shipment is officially delayed.
- **shippedDate**: `shippedDate` - `orderDate` = Order processing time
- **shipVia**: references the `shipperID` from `shippers`
- **freight**: Not sure if this is the shipping costs separately OR the weight of the shipment in kilograms or so?!
- **shipName / shipAddress / shipCity / shipRegionm / shipPostalCode / shipCountry**: Should all reference the respective columns from `customers` for the respective `customerID` of that order. 

# products.csv

In [24]:
products = pd.read_csv('./raw_data/products.csv')
print(products.head())

   productID                   productName  categoryID      quantityPerUnit  \
0          1                          Chai           1   10 boxes x 20 bags   
1          2                         Chang           1   24 - 12 oz bottles   
2          3                 Aniseed Syrup           2  12 - 550 ml bottles   
3          4  Chef Anton's Cajun Seasoning           2       48 - 6 oz jars   
4          5        Chef Anton's Gumbo Mix           2             36 boxes   

   unitPrice  unitsInStock  unitsOnOrder  reorderLevel  discontinued  
0      18.00            39             0            10             0  
1      19.00            17            40            25             0  
2      10.00            13            70            25             0  
3      22.00            53             0             0             0  
4      21.35             0             0             0             1  


- **productName**: special characters?!
- **categoryID**: references `categoryID` from `categories`
- **quantityPerUnit**: Bit of a messy dimension. We could isolate several possible sub-dimensions here such as "containerType", "containerSize", "containersPerUnit" or sometimes even "subContainer" and "subContainerSize". Example for `productID` == 1: `containerType` == 'box', `containerSize` = NULL, `containersPerUnit` == 10, `subContainer` == "bag", `subContainerSize` = NULL, `subContainersPerContainer` == 20. The main problem here is that it might be tricky to automate these derived dimensions.
- **unitPrice**: Must contain two decimal places
- **unitsInStock / unitsOnOrder**: If the latter is greater than the former, you have backorders
- **reorderLevel**: If your `unitsInStock` are below this level, you need to replenish your stock.
- **discontinued**: Seems to be binary: Has the production of this product stopped, i.e. can you restock this item? If not, some of your orders might be in danger because you might not be able to fulfil them.

# shippers.csv

In [25]:
shippers = pd.read_csv('./raw_data/shippers.csv')
print(shippers.head())

   shipperID       companyName           phone
0          1    Speedy Express  (503) 555-9831
1          2    United Package  (503) 555-3199
2          3  Federal Shipping  (503) 555-9931


- **companyName**: See discussion about special characters in proper names above.
- **phone**: See discussion about common formats for phone numbers above.